In [2]:
import json
import re
import orgelpredigt_analysis as oa
from rapidfuzz import fuzz
import pandas as pd

In [3]:
def fuzzy_match(a, b, threshold=80):
    similarity_score = fuzz.ratio(a, b)
    return similarity_score

In [4]:
def is_song_in_book(id):
    match = re.findall(r'E10[0-9]{4}', id)[0]
    
    with open('songs_to_pages_mapping.json') as f:
        songbook_pages = json.load(f)
    if songbook_pages[match]["pages"] == '':
        return False
    else:
        return True

In [5]:
def song_page(id): 
    match = re.findall(r'E10[0-9]{4}', id)[0]
    with open('songs_to_pages_mapping.json') as f:
        songbook_pages = json.load(f)
    page = songbook_pages[match]["pages"]
    
    return [int(page) + 42, int(page) + 43, int(page) + 44]

In [6]:
sermon = oa.Sermon("E000052")

In [7]:
known_song_quotes = list(set([q for q in sermon.all_references if q.startswith('E10')]))
with open('songs_to_pages_mapping.json') as f:
    songbook_pages = json.load(f)

relevant_pages = []
for i in known_song_quotes:
    page = songbook_pages[i]['pages']
    if page != '':
        page = str(int(page) + 42)          # add 42 to offset prefatory matter in scans
        add_page = str(int(page) + 1)
        add_page_2 = str(int(page) + 2)
        relevant_pages.append([i, page])
        relevant_pages.append([i, add_page, add_page_2])
print(relevant_pages)
page_nrs = []
for p in relevant_pages:
    page_nrs.append(p[1])

[['E100057', '608'], ['E100057', '609', '610'], ['E100141', '650'], ['E100141', '651', '652'], ['E100017', '434'], ['E100017', '435', '436'], ['E100067', '925'], ['E100067', '926', '927'], ['E100138', '1057'], ['E100138', '1058', '1059'], ['E100022', '740'], ['E100022', '741', '742'], ['E100132', '226'], ['E100132', '227', '228'], ['E100078', '642'], ['E100078', '643', '644'], ['E100134', '1039'], ['E100134', '1040', '1041']]


In [8]:
relevant_page_texts = []
#for n in page_nrs:
for n in range(41, 1291):
    with open(f"source_texts/praxis_pietatis_verses/{n}.json") as f:
        page = json.load(f)
    page_info = {}
    page_info[n] = page
    relevant_page_texts.append(page_info)

In [9]:
hits = []

for i in range(len(sermon.chunked)):                # for each paragraph
    for j in range(len(sermon.chunked[i])):         # for each sentence
        if " bibel" in sermon.chunked[i][j]["types"]:
            continue
        else:
            query = " ".join(sermon.chunked[i][j]["words"])
            query = re.sub(r'[/.,;:?!]', '', query)
            for page in relevant_page_texts:
                for pagenr, verses in page.items():
                    for verse in verses:
                        sim_score = fuzz.ratio(query, verse)
                        if sim_score >= 80:
                            hits.append([query, str(i) +  "-" + str(j), pagenr, verse, f"{sim_score:.2f}"])


In [10]:
guessed_hits = pd.DataFrame(hits, columns=["Predigt", "Satz", "Liederbuch", "Liedvers", "Ähnlichkeit"])
guessed_hits['Dopplung'] = guessed_hits.groupby('Satz')['Satz'].transform(lambda x: x.duplicated())

guessed_hits.sort_values(by=["Predigt"])

,Predigt,Satz,Liederbuch,Liedvers,Ähnlichkeit,Dopplung
41,alle pfeile von mir jagen,38-79,227,alle pfeile von mir jagen,100.00,False
4,auff der harff und säyten-spiel,21-28,652,auf der harff und sente friel,80.00,False
43,da der heiligen zwölff bohten zahl,47-83,642,der heiligen zwölffboten zahl,92.06,False
19,da die engel singen die nova cantica,38-23,435,denn da da die engel fingen nova cantica,81.58,False
40,damit ich den feind kan schlagen,38-78,227,damit ich den feind kan schlagen,100.00,False
8,daß uns dein nahm werd wolbekandt,24-4,608,daß uns dein nam sey wol bekannt,86.15,False
7,den glauben mehr stärck den verstand,24-3,608,den glauben mehr stärck den verstand,100.00,False
6,den mund zum lobe dein,24-1,608,deffn unsern mund zum lobe dein,83.02,False
42,denn die nacht ist vergangen,38-137,575,der sunden nacht ist vergangen,86.21,False
18,der da heilig ist,35-43,634,der da machtig ist,80.00,False


In [11]:
dopplungen = guessed_hits.loc[guessed_hits['Dopplung'] == True].sort_values(['Ähnlichkeit'], ascending=False).groupby('Satz')
dopplungen.head()

,Predigt,Satz,Liederbuch,Liedvers,Ähnlichkeit,Dopplung
30,jesu meine freude,38-55,334,jesu meine freude p,97.30,True
37,jesu meine freude,38-55,926,jefu meine freude j,91.89,True
17,der finger an gottes rechten hand,34-17,622,der finger got tes rechter hand,90.62,True
25,jesu meine freude,38-55,239,djesu meine freude seyn,87.80,True
33,jesu meine freude,38-55,579,mel jesu meine freude,87.18,True
26,jesu meine freude,38-55,280,mel jesu meine freude,87.18,True
10,heilig heilig heilig ist gott,24-6,1066,der heilig heilig heilig ist,84.21,True


In [12]:

def flatten(xss):
    return [x for xs in xss for x in xs]

In [13]:
validation = []
for i in range(len(sermon.chunked)):                # for each paragraph
    for j in range(len(sermon.chunked[i])):         # for each sentence
        if " musikwerk" in sermon.chunked[i][j]["types"]:
            line = " ".join(sermon.chunked[i][j]["words"])
            refs = ", ".join(set(flatten(sermon.chunked[i][j]["references"])))
            validation.append([line, str(i) + "-" + str(j), refs])


In [14]:
known_hits = pd.DataFrame(validation, columns=["Predigt", "Satz", "Referenz"])
known_hits = known_hits[known_hits['Referenz'].apply(is_song_in_book)]
known_hits["Ref_Seite"] = known_hits['Referenz'].apply(song_page)
known_hits

,Predigt,Satz,Referenz,Ref_Seite
0,zwingt die säyten in cythara etc.,20-18,E100022,"[740, 741, 742]"
1,daß es heist: mein hertz soll dir allzeit brin...,21-21,E100141,"[650, 651, 652]"
2,deines lobs gebührlichs theil/,21-22,E100141,"[650, 651, 652]"
3,auch soll meine zunge singen/,21-23,E100141,"[650, 651, 652]"
4,täglich dein unzählich heyl.,21-24,E100141,"[650, 651, 652]"
5,jch selbst werde wie gantz neue/,21-25,E100141,"[650, 651, 652]"
6,sing und klinge deine treue/,21-26,E100141,"[650, 651, 652]"
7,meines lebens einges ziel/,21-27,E100141,"[650, 651, 652]"
8,auff der harff und säyten-spiel.,21-28,E100141,"[650, 651, 652]"
9,thu auff/ herr jesu/,24-0,E100057,"[608, 609, 610]"


In [15]:
# Print the rows in guessed_hits that are not in known_hits
print("Rows in guessed_hits not in known_hits:")
guessed_hits[~guessed_hits['Satz'].isin(known_hits['Satz'])]

Rows in guessed_hits not in known_hits:


,Predigt,Satz,Liederbuch,Liedvers,Ähnlichkeit,Dopplung
5,steine und wilde thiere,21-75,940,wie nofund wilde thiere,82.61,False
13,so wir haben in christo jesu,25-80,1233,so wir haben ift christus selbst,80.00,False
14,der die welt überwindet,30-36,593,der die hill überwandt,80.00,False
15,wo nicht gott der heilige geist,34-8,606,mit gott dem heilgen geist,80.70,False
16,der finger an gottes rechten hand,34-17,226,der fikt zu gottes rechten hand,84.38,False
17,der finger an gottes rechten hand,34-17,622,der finger got tes rechter hand,90.62,True
18,der da heilig ist,35-43,634,der da machtig ist,80.00,False
42,denn die nacht ist vergangen,38-137,575,der sunden nacht ist vergangen,86.21,False


In [16]:
# Print the rows in known_hits that are not in guessed_hits
print("Rows in known_hits not in guessed_hits:")
known_hits[~known_hits['Satz'].isin(guessed_hits['Satz'])]

Rows in known_hits not in guessed_hits:


,Predigt,Satz,Referenz,Ref_Seite
1,daß es heist: mein hertz soll dir allzeit brin...,21-21,E100141,"[650, 651, 652]"
2,deines lobs gebührlichs theil/,21-22,E100141,"[650, 651, 652]"
3,auch soll meine zunge singen/,21-23,E100141,"[650, 651, 652]"
4,täglich dein unzählich heyl.,21-24,E100141,"[650, 651, 652]"
9,thu auff/ herr jesu/,24-0,E100057,"[608, 609, 610]"
11,bereit das hertz zur andacht fein/,24-2,E100057,"[608, 609, 610]"
14,biß wir singen mit gottes heer/,24-5,E100057,"[608, 609, 610]"
16,der herr/ und schauen ihn von angesicht/,24-7,E100057,"[608, 609, 610]"
17,in ewger freud und seelgem licht.,24-8,E100057,"[608, 609, 610]"
20,daß ich möge mit jesulein dem allerschönsten b...,25-33,E100022,"[740, 741, 742]"


In [17]:
merged_df = pd.merge(known_hits, guessed_hits, on='Satz', how='outer')
merged_df = merged_df.drop("Predigt_y", axis=1)
merged_df['Liederbuch'] = merged_df['Liederbuch'].astype('Int64')
merged_df = merged_df.rename(columns={"Predigt_x": "Predigt"})
merged_df

,Predigt,Satz,Referenz,Ref_Seite,Liederbuch,Liedvers,Ähnlichkeit,Dopplung
0,zwingt die säyten in cythara etc.,20-18,E100022,"[740, 741, 742]",741,zwingt die sensen in cythara,83.33,False
1,daß es heist: mein hertz soll dir allzeit brin...,21-21,E100141,"[650, 651, 652]",<NA>,NaN,NaN,NaN
2,deines lobs gebührlichs theil/,21-22,E100141,"[650, 651, 652]",<NA>,NaN,NaN,NaN
3,auch soll meine zunge singen/,21-23,E100141,"[650, 651, 652]",<NA>,NaN,NaN,NaN
4,täglich dein unzählich heyl.,21-24,E100141,"[650, 651, 652]",<NA>,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
63,die theuren märtrer allzumahl dich loben herr ...,47-85,E100078,"[642, 643, 644]",<NA>,NaN,NaN,NaN
64,mit den seraphinen prangen/,47-87,E100134,"[1039, 1040, 1041]",1040,da die seraphinen prangen,86.27,False
65,und das hohe lied anfangen:,47-88,E100134,"[1039, 1040, 1041]",1040,und das hohe lied anfangen heilig,88.14,False
66,heilig/ heilig/ heilig heist gott der vater/,47-89,E100134,"[1039, 1040, 1041]",1040,heilig heilig heißt ott der vater,86.49,False


In [18]:
converged_df = pd.merge(known_hits, guessed_hits, on='Satz', how='inner')
converged_df

,Predigt_x,Satz,Referenz,Ref_Seite,Predigt_y,Liederbuch,Liedvers,Ähnlichkeit,Dopplung
0,zwingt die säyten in cythara etc.,20-18,E100022,"[740, 741, 742]",zwingt die säyten in cythara etc,741,zwingt die sensen in cythara,83.33,False
1,jch selbst werde wie gantz neue/,21-25,E100141,"[650, 651, 652]",jch selbst werde wie gantz neue,652,ich selbst werde wie ganz nene,91.80,False
2,sing und klinge deine treue/,21-26,E100141,"[650, 651, 652]",sing und klinge deine treue,652,sing und klinge deine treue,100.00,False
3,meines lebens einges ziel/,21-27,E100141,"[650, 651, 652]",meines lebens einges ziel,652,mei nes lebens einges ziel,98.04,False
4,auff der harff und säyten-spiel.,21-28,E100141,"[650, 651, 652]",auff der harff und säyten-spiel,652,auf der harff und sente friel,80.00,False
5,den mund zum lobe dein/,24-1,E100057,"[608, 609, 610]",den mund zum lobe dein,608,deffn unsern mund zum lobe dein,83.02,False
6,den glauben mehr/ stärck den verstand/,24-3,E100057,"[608, 609, 610]",den glauben mehr stärck den verstand,608,den glauben mehr stärck den verstand,100.00,False
7,daß uns dein nahm werd wolbekandt/,24-4,E100057,"[608, 609, 610]",daß uns dein nahm werd wolbekandt,608,daß uns dein nam sey wol bekannt,86.15,False
8,"heilig/ heilig/ heilig ist gott,",24-6,E100057,"[608, 609, 610]",heilig heilig heilig ist gott,638,und heilig heilig heilig singt,81.36,False
9,"heilig/ heilig/ heilig ist gott,",24-6,E100057,"[608, 609, 610]",heilig heilig heilig ist gott,1066,der heilig heilig heilig ist,84.21,True


In [19]:
converged_df = pd.merge(known_hits, guessed_hits, on='Satz', how='inner')
converged_df["in_page_list"]  = converged_df.apply(lambda row: row['Liederbuch'] in row['Ref_Seite'], axis=1)
agreed_hits = converged_df["in_page_list"].value_counts()[True]
divergent_hits = converged_df["in_page_list"].value_counts()[False]
print(f"Number of sentences that were identified on the same page: {agreed_hits}")
print(f"Number of sentences that were identified on different pages: {divergent_hits}")

Number of sentences that were identified on the same page: 23
Number of sentences that were identified on different pages: 17


In [20]:
doublings = converged_df[(converged_df["in_page_list"] == converged_df["Dopplung"])]
merged_df = merged_df.drop("Predigt_y", axis=1)
doublings

KeyError: "['Predigt_y'] not found in axis"